# Rought plan

1. Set up path to place files
2. get the source_directory to search for the files
3. os.walk() to find the files
4. open the file
   1. move pointer to the end of file if not using "a+" mode
   2. normal termination line
      1. if no normal termination line break and move on to next file
   3. move pointer to the top of the file
   4. iterate using readline
      1. startswith("%"), grab line
      2. startswith("#") grab line and all lines after until reach "------" line
   5. end


# Locate/setup directory

In [64]:
import os

def set_data_path() -> str:
    outfile_name = input("Name the file to store the data:\n(Do not include the extension)\n")

    # Locate current working directory as destination for report
    data_path = os.path.join(os.getcwd(),outfile_name)

    while True :
        # Two answers: acceptable: 1. path, 2. "", all else are bad"
        answer = str(input("The data recorded will be at path:\n%10s\nHit enter to proceed, or type new path without '/LogData.csv':\n  " % data_path))
        if answer == "" :
            break
        elif os.path.isdir(answer) :
            data_path = os.path.join(answer,outfile_name)
            break

    return data_path

def test() :
    """
    Simple data file string is:
            LogFile

    Copy and past path:
            /Users/Riley/Documents/Research/User_Input_Module/
    """
    data_path = set_data_path()
    print(data_path)

test()

/Users/Riley/Documents/Research/User_Input_Module/LogFile


In [65]:
import pprint as pp
def find_files_in_dir() -> list[str] :
    # Get valid source directory from user
    source_dir = input("Please input the directory path to search:\n")
    while not os.path.isdir(source_dir) :
        source_dir = input("Invalid path, enter a new directory path to search:\n")

    # Create a list of all the log files in the directory
    listoffiles = list()
    for (dirpath, dir_names, filenames) in os.walk(source_dir):
        listoffiles += [os.path.join(dirpath,file) for file in filenames if file.split(".")[len(file.split("."))-1] == "log"]

    print("%d log files were found in the directory branch:\n%s" % (len(listoffiles), source_dir))
    return listoffiles

def test():
    """
    Use example files in the directory at the path:
            /Users/Riley/Documents/Research/User_Input_Module/Gaussian_Outputs_Keywords/
    """
    file_list = find_files_in_dir()
    pp.pprint(file_list)

test()

6 log files were found in the directory branch:
/Users/Riley/Documents/Research/User_Input_Module/Gaussian_Outputs_Keywords/
['/Users/Riley/Documents/Research/User_Input_Module/Gaussian_Outputs_Keywords/AllylChlorideClTS.log',
 '/Users/Riley/Documents/Research/User_Input_Module/Gaussian_Outputs_Keywords/NotNormalTerm.log',
 '/Users/Riley/Documents/Research/User_Input_Module/Gaussian_Outputs_Keywords/EmptyFile.log',
 '/Users/Riley/Documents/Research/User_Input_Module/Gaussian_Outputs_Keywords/Pd2p-ene-CO-transcis-P-fromciscis.log',
 '/Users/Riley/Documents/Research/User_Input_Module/Gaussian_Outputs_Keywords/Cl.log',
 '/Users/Riley/Documents/Research/User_Input_Module/Gaussian_Outputs_Keywords/AllylChloride.log']


# Handling the file challenge
open the file
   1. move pointer to the end of file if not using "a+" mode
   2. normal termination line
      1. if no normal termination line break and move on to next file
   3. move pointer to the top of the file
   4. iterate using readline
      1. startswith("%"), grab line
      2. startswith("#") grab line and all lines after until reach "------" line
   5. end

In [66]:
from asyncore import file_wrapper

def check_log_file(file_path: str, file_obj: file_wrapper) -> bool:
    """
    Check log file searches the file object for the line
    "Normal termination of Gaussian 16"

    This is usually a substring of the last line in the log file, if it terminates correctly.
    Sometimes with multistep calculations, there will be a "Normal termination of Gaussian 16"
    line for every calculation step, or some may contain a "Error termination of Gaussian 16."

    We only want to record the keywords from the files that have terminated normally.
    Those keywords are deemed as valid by the Gaussian 16 software.
    Therefore, the log file must pass this check if we are going to count its keywords.

    This function iterates over the bytes/characters of the file in reverse order until it finds the line
    "Normal termination of Gaussian 16." Keep in mind that if the file contains multi-step calculations,
    and one of those calculations finished successfully, then the keywords will be counted.
    """
    check = True
    empty = False

    file_name = (file_path.split("/"))[len(file_path.split("/"))-1]

    # Check if size of file is 0
    if os.stat(file_path).st_size == 0 :
        empty = True
        check = False

    if empty != True :
        # Move the cursor to the end of the file
        file_obj.seek(0, os.SEEK_END)

        # Get the current position of the pointer
        pointer_location = file_obj.tell()
        
        # Create a buffer to keep the last read line
        buffer = bytearray()

        #Loop in reverse order until "Normal termination" line is found
        while pointer_location >= 0 :
            # Move the file pointer to the location pointed by the pointer
            file_obj.seek(pointer_location)

            pointer_location -= 1 # Shift pointer location back by 1
            
            new_byte = file_obj.read(1) # Read that byte/character

            if new_byte == b'\n' : # If the byte is a newline character, then the end of a line has been reached
                line = buffer.decode()[::-1] #Fetch the line from buffer
                if 'Normal termination of Gaussian 16' in line :
                    print("File, %s, terminated normally." % file_name)
                    break
                # Reinitialize the byte array to save next line
                buffer = bytearray()
            else:
                buffer.extend(new_byte)

        # As file is read completely, if the line has not been detected then the file did not terminate normally.
        if pointer_location <= 0:
            print("Normal termination not detected in %s." %file_name)
            check = False
    else:
        print("File, %s, is empty." % file_name)

    return check

def test():

    # file_path1 is a perfectly completed calculation file
    file_path1 = '/Users/Riley/Documents/Research/User_Input_Module/Gaussian_Outputs_Keywords/AllylChloride.log'
    file_path2 = '/Users/Riley/Documents/Research/User_Input_Module/Gaussian_Outputs_Keywords/AllylChlorideClTS.log'

    # file_path2 is a multi-step calculation log file. Both calculations terminated with error
    file_path3 = '/Users/Riley/Documents/Research/User_Input_Module/Gaussian_Outputs_Keywords/NotNormalTerm.log'
    # file_path3 is an empty file with a log extension
    file_path4 = '/Users/Riley/Documents/Research/User_Input_Module/Gaussian_Outputs_Keywords/EmptyFile.log'
    file_list = [file_path1,file_path2,file_path3, file_path4]
    
    checks = list()
    for file_path in file_list :
        with open(file_path,"rb") as file_obj :
            checks.append(check_log_file(file_path,file_obj))
    
    [print("    File:\n     %s\n     Normal termination: %s" %(file,checks[index])) for index, file in enumerate(file_list)]

test()

File, AllylChloride.log, terminated normally.
File, AllylChlorideClTS.log, terminated normally.
Normal termination not detected in NotNormalTerm.log.
File, EmptyFile.log, is empty.
    File:
     /Users/Riley/Documents/Research/User_Input_Module/Gaussian_Outputs_Keywords/AllylChloride.log
     Normal termination: True
    File:
     /Users/Riley/Documents/Research/User_Input_Module/Gaussian_Outputs_Keywords/AllylChlorideClTS.log
     Normal termination: True
    File:
     /Users/Riley/Documents/Research/User_Input_Module/Gaussian_Outputs_Keywords/NotNormalTerm.log
     Normal termination: False
    File:
     /Users/Riley/Documents/Research/User_Input_Module/Gaussian_Outputs_Keywords/EmptyFile.log
     Normal termination: False


# Extracting the data from the log file

> The file has already been read as a binary file
> 
>   - This was to check the last lines to see if it contained "Normal termination of Gaussian 16"
> 
>   - There really isn't a need to read binary, unless that is required for reading the last line.
> 
> For extracting the keywords, intuitively, it could also be read character by character. The difficulty would be making sure the characters don't get combined all together
> 
> It seems easier to read by line. 
> 
>   - The challenge of reading line by line is if the Route section spills over into two lines, then its possible one of the keywords will get messed up. 
> 
>   - This multiplies one keyword messed up for every line spilled over. So for every line spilled over there are two bad inputs. 
> 
>   - Ideally, there won't be that many, so these will be relatively low scoring keywords when counting them as popular data.
> 
> The next task is to compile all of the keywords into a dictionary that counts them. Then to sort the dictionary by the values, and write it into an outfile.

In [67]:
def extract_data(file_obj: file_wrapper) -> tuple[list[str]]:
    # Initialize local var
    link0 = bytearray()
    route_cmds = bytearray()

    # Move the cursor to the beginning of the file
    file_obj.seek(0)

    # Read first line
    line = file_obj.readline()
    while line:
        # if the line starts with a %, it is a link0 command
        if b"%" in line[:3] :
                #print("Found link0 input\n%s" % line)
                line = line.split()
                [link0.extend(b" " + elem) for elem in line if b"%" in elem and elem != b""]
        # if the line starts with a #, it's the route section line
        elif b"#" in line[:3] :
            #print("Found Route Line\n%s" % line)
            line = line.split()
            [route_cmds.extend(b" " + elem) for elem in line if b"#" not in elem and elem != b""]
            # grab next line if the contents spill over
            line = file_obj.readline()
            while line :
                if b'----------------------------------------------------------------------' in line :
                    break
                line = line.split()
                [route_cmds.extend(b" " + elem) for elem in line if elem != b""]
                line = file_obj.readline()
            break
        line = file_obj.readline()

    return link0.decode("ascii").split(), route_cmds.decode("ascii").split()


def test():
    # file_path1 is a perfectly completed calculation file
    file_path1 = '/Users/Riley/Documents/Research/User_Input_Module/Gaussian_Outputs_Keywords/AllylChloride.log'
    file_path2 = '/Users/Riley/Documents/Research/User_Input_Module/Gaussian_Outputs_Keywords/AllylChlorideClTS.log'

    # file_path2 is a multi-step calculation log file. Both calculations terminated with error
    file_path3 = '/Users/Riley/Documents/Research/User_Input_Module/Gaussian_Outputs_Keywords/NotNormalTerm.log'
    # file_path3 is an empty file with a log extension
    file_path4 = '/Users/Riley/Documents/Research/User_Input_Module/Gaussian_Outputs_Keywords/EmptyFile.log'
    file_list = [file_path1,file_path2,file_path3, file_path4]

    for file_path in file_list :
        with open(file_path,"rb") as infile :
            check = check_log_file(file_path=file_path,file_obj=infile)
            if check : 
                (link0, route_cmds) = extract_data(file_obj=infile)
                print(link0)
                print(route_cmds)
                print()
            else:
                pass

test()

File, AllylChloride.log, terminated normally.
['%chk=AllylChloride.chk', '%nprocs=60', '%mem=60GB']
['opt=(maxcycles=85)', 'freq=noraman', 'hf/3-21g', 'scrf=(smd,solvent=dmso)', 'geo', 'm=connectivity']

File, AllylChlorideClTS.log, terminated normally.
['%oldchk=AllylChlorideClTS.chk', '%chk=AllylChlorideClTS-TS.chk']
['opt=(ts,noeigen,readfc,modredundant)', 'freq', 'hf/3-21g', 'scrf=(smd,solvent', '=dmso)', 'nosymm', 'geom=checkpoint', 'guess=read']

Normal termination not detected in NotNormalTerm.log.
File, EmptyFile.log, is empty.


In [68]:
def update_mostFreq_data(mostFreq_dict:dict[str:int], keywords:list[str]) -> None:
    """ 
    This function updates the mostFreq data used to track the most popular G16 keywords:
    Update the count if the keyword is in the dictionary.
    If the keyword is not in the dictionary, its added with a default count of 1.
    """
    [mostFreq_dict.update({elem:mostFreq_dict[elem]+1}) for elem in keywords if elem in mostFreq_dict.keys()]
    [mostFreq_dict.setdefault(elem,1) for elem in keywords if elem not in mostFreq_dict.keys()]

def test():
    # Data output by the extract data function
    link0 = ['%oldchk=AllylChlorideClTS.chk', '%chk=AllylChlorideClTS-TS.chk', '%nprocs=60', '%mem=60GB', '%chk=AllylChloride.chk']
    route_cmds = ['opt=(ts,noeigen,readfc,modredundant)', 'freq', 'hf/3-21g', 'scrf=(smd,solvent=dmso)', 'nosymm', 'geom=checkpoint', 'guess=read']

    # Initializing variables for the data. These need to be either global variables or class variables, 
    # so all the files update the same dictionary
    link0_mostFreq = dict()
    route_mostFreq = dict()

    # Example link0 keyword update
    pp.pprint(link0_mostFreq)
    update_mostFreq_data(link0_mostFreq,link0)
    pp.pprint(link0_mostFreq)
    
    # Example Route Section Keyword Update
    pp.pprint(route_mostFreq)
    update_mostFreq_data(route_mostFreq,route_cmds)
    pp.pprint(route_mostFreq)

test()

{}
{'%chk=AllylChloride.chk': 1,
 '%chk=AllylChlorideClTS-TS.chk': 1,
 '%mem=60GB': 1,
 '%nprocs=60': 1,
 '%oldchk=AllylChlorideClTS.chk': 1}
{}
{'freq': 1,
 'geom=checkpoint': 1,
 'guess=read': 1,
 'hf/3-21g': 1,
 'nosymm': 1,
 'opt=(ts,noeigen,readfc,modredundant)': 1,
 'scrf=(smd,solvent=dmso)': 1}
